# Import libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pip install diptest

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 2.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
from sklearn.metrics import normalized_mutual_info_score as nmi
import torch
import torchvision
import torchvision.datasets as datasets
import diptest

# Math functions

In [ ]:
def squared_euclidean_distance(centers, embedded_data):
    tensor1 = centers.unsqueeze(0) #add one dim in front
    tensor2 = embedded_data.unsqueeze(1) #add one dim in last (same as .unsqueeze(-1))
    sed = (tensor1 - tensor2).pow(2).sum(2)  #power by 2 -> sum along the axis 2
    return sed

In [ ]:
def z_score(data):
    return (data - np.mean(data)) / np.std(data)

In [ ]:
def int_to_one_hot(label_tensor, n_labels):
    onehot = torch.zeros([label_tensor.shape[0], n_labels], dtype=torch.float, device=label_tensor.device)
    onehot.scatter_(1, label_tensor.unsqueeze(1).long(), 1.0)
    return onehot

# Datasets

Image datasets

In [ ]:
def data_preprocess(train, test):
  data = torch.cat([train.data, test.data], dim=0) #torch.Size([60000+10000, 28, 28]) for MNIST
  labels = torch.cat([train.targets, test.targets], dim=0)

  if data.dim() == 3:
    data = data.reshape(-1, data.shape[1] * data.shape[2]) #flatten 28 x 28 images into 784 array (one dim vector) for MNIST
  else:
    data = data.reshape(-1, data.shape[1] * data.shape[2] * data.shape[3])

  #move data to CPU, convert to numpy, and set requires_grad = False
  data = data.detach().cpu().numpy()
  labels = labels.detach().cpu().numpy()

  data = z_score(data) #Z-score normalization

  return data, labels

In [ ]:
def MNIST():
  trainset = datasets.MNIST(root='./data', train=True, download=True)
  testset = datasets.MNIST(root='./data', train=False, download=True)

  data, labels = data_preprocess(trainset, testset)

  return data, labels

In [ ]:
def FashionMNIST():
  trainset = datasets.FashionMNIST(root='./data', train=True, download=True)
  testset = datasets.FashionMNIST(root='./data', train=False, download=True)
  
  data, labels = data_preprocess(trainset, testset)

  return data, labels

In [ ]:
def KMNIST():
  trainset = datasets.KMNIST(root='./data', train=True, download=True)
  testset = datasets.KMNIST(root='./data', train=False, download=True)
  
  data, labels = data_preprocess(trainset, testset)

  return data, labels

In [ ]:
def USPS():
  trainset = datasets.USPS(root='./data', train=True, download=True)
  testset = datasets.USPS(root='./data', train=False, download=True)
  
  data = np.r_[trainset.data, testset.data]
  labels = np.r_[trainset.targets, testset.targets]
  data = data.reshape(-1, 256)

  data = z_score(data)

  return data, labels

In [ ]:
def load_data(filename):
  dataset = np.genfromtxt(filename, delimiter=",")
  return dataset[:, :-1], dataset[:, -1]

In [ ]:
def OPTDIGITS():
  trainset = load_data('optdigits.tra')
  testset = load_data('optdigits.tes')

  data = np.r_[trainset[0], testset[0]]
  labels = np.r_[trainset[1], testset[1]]

  data = z_score(data)
  
  return data, labels

In [ ]:
def PENDIGITS():
  trainset = load_data('pendigits.tra')
  testset = load_data('pendigits.tes')

  data = np.r_[trainset[0], testset[0]]
  labels = np.r_[trainset[1], testset[1]]

  data = scale(data, axis=0)

  return data, labels

# Dip-test

Test

In [ ]:
# generate some bimodal random draws
N = 1000
hN = N // 2
x = np.empty(N, dtype=np.float64)
x[:hN] = np.random.normal(0.4, 1.0, hN)
x[hN:] = np.random.normal(-0.4, 1.0, hN)

# only the dip statistic
dip = diptest.dipstat(x)

# both the dip statistic and p-value
dip, pval = diptest.diptest(x)

# AutoEncoder

In [ ]:
def torch_device():
  return torch.device('cuda' if torch.cuda.is_available() else 'cpu') #return device

In [ ]:
def encode_batch(dataloader, autoenc, device): #embed the dataset in a mini-batch fashion
    embeddings = [autoenc.encode(batch.to(device)).detach().cpu() for batch in dataloader]
    return torch.cat(embeddings, dim=0).numpy()

In [ ]:
def nearest_points_to_optimal_centers(X, optimal_centers, embedded_data):
    best_center_points = np.argmin(cdist(optimal_centers, embedded_data), axis=1) #eq.2 
    return X[best_center_points, :], embedded_data[best_center_points, :]

In [ ]:
def nearest_points(large_cluster, center, small_cluster, max_diff_factor, min_size):
    nearest_points = np.argsort(cdist(large_cluster, [center]), axis=0)
    sample_size = small_cluster * max_diff_factor

    if small_cluster + sample_size < min_size: # Check if more points should be taken because the other cluster is too small
        sample_size = min(min_size - small_cluster, len(large_cluster))

    return large_cluster[nearest_points[:sample_size, 0]]

In [ ]:
def iterate_over_batches(iteration, train_set, device, autoencoder, centers_torch, loss_fn, labels_torch, c_current, matrix_final, loss_weight, optimizer):
  for batch, ids in train_set:  # Iterate over batches
      batch_data = batch.to(device)
      embedded = autoencoder.encode(batch_data)
      reconstruction = autoencoder.decode(embedded)
      embedded_centers_torch = autoencoder.encode(centers_torch)
      ae_loss = loss_fn(reconstruction, batch_data) # Reconstruction Loss
      squared_diffs = squared_euclidean_distance(embedded_centers_torch, embedded) # Get distances between points and centers. Get nearest center
      current_labels = squared_diffs.argmin(1) if iteration != 0 else labels_torch[ids]
      escaped_diffs = torch.matmul(int_to_one_hot(current_labels, c_current).float(), matrix_final) * squared_diffs
      squared_center_diffs = squared_euclidean_distance(embedded_centers_torch, embedded_centers_torch) # Normalize loss by cluster distances
      mask = torch.where(squared_center_diffs != 0) # Ignore zero values (diagonal)
      sqrt_masked_center_diffs = squared_center_diffs[mask[0], mask[1]].sqrt()
      masked_center_diffs_std = sqrt_masked_center_diffs.std() if len(sqrt_masked_center_diffs) > 2 else 0
      cluster_loss = escaped_diffs.sum(1).mean() * (1 + masked_center_diffs_std) / sqrt_masked_center_diffs.mean()
      loss = ae_loss + (cluster_loss * loss_weight)# Loss function
      optimizer.zero_grad()
      loss.backward()       # Backward pass
      optimizer.step()
  return ae_loss, cluster_loss, loss

In [ ]:
def dip_deck_training(X, c_current, threshold, loss_weight, centers_cpu, labels_cpu, matrix_cpu, c_max, c_min, d_epochs, optimizer, loss_fn, autoencoder, device, train_set, test_set, diff_factor, debug):
    i = 0
    while i < d_epochs:
        labels_torch = torch.from_numpy(labels_cpu).long().to(device)
        centers_torch = torch.from_numpy(centers_cpu).float().to(device)
        matrix_torch = torch.from_numpy(matrix_cpu).float().to(device)
        dip_matrix_eye = matrix_torch + torch.eye(c_current, device=device)# Get dip costs matrix
        
        ae_loss, cluster_loss, loss = iterate_over_batches(i, train_set, device, autoencoder, centers_torch, loss_fn, labels_torch, c_current, dip_matrix_eye / dip_matrix_eye.sum(1).reshape((-1, 1)), loss_weight, optimizer)
        
        # labels_cpu = labels_torch.detach().cpu().numpy()
        embedded_data = encode_batch(test_set, autoencoder, device)     # Update centers
        embedded_centers_cpu = autoencoder.encode(centers_torch).detach().cpu().numpy()
        labels_cpu = np.argmin(cdist(embedded_centers_cpu, embedded_data), axis=0)
        optimal_centers = np.array([np.mean(embedded_data[labels_cpu == cluster_id], axis=0) for cluster_id in range(c_current)])
        centers_cpu, embedded_centers_cpu = nearest_points_to_optimal_centers(X, optimal_centers, embedded_data)
        matrix_cpu = dip_matrix(embedded_data, embedded_centers_cpu, labels_cpu, c_current, diff_factor) # Update Dips

        if debug:
          print("Iteration {0}  (n_clusters = {4}) - reconstruction loss: {1} / cluster loss: {2} / total loss: {3}".format(i, ae_loss.item(), cluster_loss.item(), loss.item(), c_current) + "\nmax dip", np.max(matrix_cpu), " at ", np.unravel_index(np.argmax(matrix_cpu, axis=None), matrix_cpu.shape))
        
        i += 1 # i is increased here. Else next iteration will start with i = 1 instead of 0 after a merge
        dip_argmax = np.unravel_index(np.argmax(matrix_cpu, axis=None), matrix_cpu.shape) # Start merging procedure
        if i != 0: # Is merge possible?
            while matrix_cpu[dip_argmax] >= threshold and c_current > c_min:
                if debug:
                  print("Start merging in iteration {0}.\nMerging clusters {1} with dip value {2}.".format(i, dip_argmax, matrix_cpu[dip_argmax]))
                i = 0 # Reset iteration and reduce number of cluster
                c_current -= 1
                labels_cpu, centers_cpu, embedded_centers_cpu, matrix_cpu = merge_by_dip_value(X, embedded_data, labels_cpu, dip_argmax, c_current, centers_cpu, embedded_centers_cpu, diff_factor)
                dip_argmax = np.unravel_index(np.argmax(matrix_cpu, axis=None), matrix_cpu.shape)
        if c_current == 1:
            if debug:
              print("Only one cluster left")
            break
    return labels_cpu, c_current, centers_cpu, autoencoder

In [ ]:
def check_clusters_sizes_diff(i, j, points_in_i, points_in_j, points_in_i_or_j, dip_centers, center_diff, diff_factor, dip_p_value, min_sample_size):
  if points_in_i.shape[0] > points_in_j.shape[0] * diff_factor or points_in_j.shape[0] > points_in_i.shape[0] * diff_factor:
      if points_in_i.shape[0] > points_in_j.shape[0] * diff_factor:
          points_in_i = nearest_points(points_in_i, dip_centers[j], points_in_j.shape[0], diff_factor, min_sample_size)
      elif points_in_j.shape[0] > points_in_i.shape[0] * diff_factor:
          points_in_j = nearest_points(points_in_j, dip_centers[i], points_in_i.shape[0], diff_factor, min_sample_size)
      
      points_in_i_or_j = np.append(points_in_i, points_in_j, axis=0)
      proj_points = np.dot(points_in_i_or_j, center_diff)
      _, dip_p_value_2 = diptest.diptest(proj_points)
      dip_p_value = min(dip_p_value, dip_p_value_2)
  return dip_p_value

In [ ]:
def dip_matrix(data, dip_centers, dip_labels, n_clusters, diff_factor=2, min_sample_size=100):
    dip_matrix = np.zeros((n_clusters, n_clusters))    # Loop over all combinations of centers
    for i in range(0, n_clusters - 1):
        for j in range(i + 1, n_clusters):
            center_diff = dip_centers[i] - dip_centers[j]
            
            points_in_i = data[dip_labels == i]
            points_in_j = data[dip_labels == j]
            points_in_i_or_j = np.append(points_in_i, points_in_j, axis=0)
            
            proj_points = np.dot(points_in_i_or_j, center_diff)
            _, dip_p_value = diptest.diptest(proj_points)
            # Check if clusters sizes differ heavily
            dip_p_value = check_clusters_sizes_diff(i, j, points_in_i, points_in_j, points_in_i_or_j, dip_centers, center_diff, diff_factor, dip_p_value, min_sample_size)
            # Add pval to dip matrix
            dip_matrix[i][j] = dip_p_value
            dip_matrix[j][i] = dip_p_value
    return dip_matrix

In [ ]:
def update_labels(labels_cpu, dip_argmax, c_current):
  for j, l in enumerate(labels_cpu):
    if l == dip_argmax[0] or l == dip_argmax[1]:
        labels_cpu[j] = c_current - 1
    elif l < dip_argmax[0] and l < dip_argmax[1]:
        labels_cpu[j] = l
    elif l > dip_argmax[0] and l > dip_argmax[1]:
        labels_cpu[j] = l - 2
    else:
        labels_cpu[j] = l - 1
  return labels_cpu

In [ ]:
def merge_by_dip_value(X, embedded_data, labels_cpu, dip_argmax, c_current, centers_cpu, embedded_centers_cpu, diff_factor):
    points_in_center_1 = len(labels_cpu[labels_cpu == dip_argmax[0]])     # Get points in clusters
    points_in_center_2 = len(labels_cpu[labels_cpu == dip_argmax[1]])
    labels_cpu = update_labels(labels_cpu, dip_argmax, c_current)# update labels
    # Find new center position
    optimal_new_center = (embedded_centers_cpu[dip_argmax[0]] * points_in_center_1 + embedded_centers_cpu[dip_argmax[1]] * points_in_center_2) / (points_in_center_1 + points_in_center_2) 
    new_center_cpu, new_embedded_center_cpu = nearest_points_to_optimal_centers(X, [optimal_new_center], embedded_data)
    centers_cpu = np.append(np.delete(centers_cpu, dip_argmax, axis=0), new_center_cpu, axis=0)  # Remove the two old centers and add the new one
    embedded_centers_cpu = np.append(np.delete(embedded_centers_cpu, dip_argmax, axis=0), new_embedded_center_cpu, axis=0)
    dip_matrix_cpu = dip_matrix(embedded_data, embedded_centers_cpu, labels_cpu, c_current, diff_factor) # Update dip values
    return labels_cpu, centers_cpu, embedded_centers_cpu, dip_matrix_cpu

In [ ]:
def train_autoencoder(trainloader, learning_rate, n_epochs, device, optimizer, loss_fn, input_dim, embedding_size, autoencoder_class):
    if embedding_size > input_dim:
      embedding_size = input_dim
    autoencoder = autoencoder_class(input_dim=input_dim, embedding_size=embedding_size).to(device) #pretrain Autoencoder
    optimizer = optimizer(autoencoder.parameters(), lr=learning_rate)
    autoencoder.start_training(trainloader, n_epochs, device, optimizer, loss_fn)
    return autoencoder

In [ ]:
class Autoencoder(torch.nn.Module):
   def __init__(self, input_dim, embedding_size):
      super(Autoencoder, self).__init__()

      self.encoder = torch.nn.Sequential(
         torch.nn.Linear(input_dim, 500), #input_dim = d
         torch.nn.LeakyReLU(inplace=True),
         torch.nn.Linear(500, 500),
         torch.nn.LeakyReLU(inplace=True),
         torch.nn.Linear(500, 2000),
         torch.nn.LeakyReLU(inplace=True),
         torch.nn.Linear(2000, embedding_size), #embedding = m
      )
      #reverse of encoder
      self.decoder = torch.nn.Sequential(
         torch.nn.Linear(embedding_size, 2000),
         torch.nn.LeakyReLU(inplace=True),
         torch.nn.Linear(2000, 500),
         torch.nn.LeakyReLU(inplace=True),
         torch.nn.Linear(500, 500),
         torch.nn.LeakyReLU(inplace=True),
         torch.nn.Linear(500, input_dim),
      )

   def encode(self, x):
     return self.encoder(x)

   def decode(self, encoded):
     return self.decoder(encoded)

   def forward(self, x):
      encoded = self.encoder(x)
      decoded = self.decoder(encoded)
      return decoded

   def start_training(self, trainloader, n_epochs, device, optimizer, loss_fn):
      for _ in range(n_epochs):
        for batch, _ in trainloader:
          batch_data = batch.to(device)
          decoded = self.forward(batch_data)
          # Compute the loss 
          loss = loss_fn(decoded, batch_data)
          # Zero your gradients for every batch
          optimizer.zero_grad()
          # Compute the gradients of the loss
          loss.backward()
          # Adjust learning weights
          optimizer.step()

# DipDeck function

In [ ]:
def handeling_errors(c_start, c_max, c_min):
    if c_max < c_min:
        raise Exception("n_clusters_max can not be smaller than n_clusters_min")
    if c_min <= 0:
        raise Exception("n_clusters_min must be greater than zero")
    if c_start < c_min:
        raise Exception("n_clusters can not be smaller than n_clusters_min")
    return None

In [ ]:
def data_loader(X, batch_size):
  train_set = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(*(torch.from_numpy(X).float(), torch.arange(0, X.shape[0]))),
                                          batch_size=batch_size, # sample random mini-batches from the data
                                          shuffle=True,
                                          drop_last=False) # create a Dataloader to test the autoencoder in mini-batch fashion (Important for validation)
  test_set = torch.utils.data.DataLoader(torch.from_numpy(X).float(),
                                          batch_size= batch_size,
                                          shuffle=False, # Note that we deactivate the shuffling
                                          drop_last=False)
  return train_set, test_set

In [ ]:
def dip_deck(X, c_start, threshold, loss_weight, c_max, c_min, batch_size, learning_rate, pt_epochs, d_epochs, optimizer, loss_fn, autoencoder, embedding_size, max_diff_factor, debug):
    handeling_errors(c_start, c_max, c_min)
    device = torch_device()
    trainloader, testloader = data_loader(X, batch_size)

    if autoencoder is None:
        autoencoder = train_autoencoder(trainloader, learning_rate, pt_epochs, device, optimizer, loss_fn, X.shape[1], embedding_size, Autoencoder)
    
    # Execute kmeans in embedded space - initial clustering
    embedded_data = encode_batch(testloader, autoencoder, device)
    kmeans = KMeans(n_clusters= c_start)
    kmeans.fit(embedded_data)

    centers_cpu, embedded_centers_cpu = nearest_points_to_optimal_centers(X, kmeans.cluster_centers_, embedded_data)  # Get nearest points to optimal centers

    c_labels_cpu, c_current, centers_cpu, autoencoder = dip_deck_training(X, c_start,     # Start training
                                                                          threshold,
                                                                          loss_weight,
                                                                          centers_cpu,
                                                                          kmeans.labels_,
                                                                          dip_matrix(embedded_data, embedded_centers_cpu, kmeans.labels_, c_start, max_diff_factor), # Initial dip values
                                                                          c_max, c_min, d_epochs,
                                                                          optimizer(autoencoder.parameters(), lr= learning_rate * 0.1),
                                                                          loss_fn, autoencoder, device, 
                                                                          trainloader, testloader,
                                                                          max_diff_factor, debug)
    return c_labels_cpu, c_current, centers_cpu, autoencoder

In [ ]:
class DipDeck():

  def __init__(self, n_clusters_start=35, dip_merge_threshold=0.9, cluster_loss_weight=1, n_clusters_max=np.inf, n_clusters_min=1, batch_size=256, learning_rate=1e-3, 
               pretrain_epochs=100, dedc_epochs=50, optimizer_class=torch.optim.Adam, loss_fn=torch.nn.MSELoss(), autoencoder=None, embedding_size=5, max_cluster_size_diff_factor=2, debug=False):
        
        self.n_clusters_start = n_clusters_start
        self.dip_merge_threshold = dip_merge_threshold
        self.cluster_loss_weight = cluster_loss_weight
        self.n_clusters_max = n_clusters_max
        self.n_clusters_min = n_clusters_min
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.pretrain_epochs = pretrain_epochs
        self.dedc_epochs = dedc_epochs
        self.optimizer_class = optimizer_class
        self.loss_fn = loss_fn
        self.autoencoder = autoencoder
        self.embedding_size = embedding_size
        self.max_cluster_size_diff_factor = max_cluster_size_diff_factor
        self.debug = debug
    

  def fit(self, X):
        labels, n_clusters, centers, autoencoder = dip_deck(X, self.n_clusters_start, self.dip_merge_threshold,
                                                             self.cluster_loss_weight, self.n_clusters_max,
                                                             self.n_clusters_min, self.batch_size, self.learning_rate,
                                                             self.pretrain_epochs, self.dedc_epochs,
                                                             self.optimizer_class,
                                                             self.loss_fn, self.autoencoder, self.embedding_size,
                                                             self.max_cluster_size_diff_factor, self.debug)
        self.labels_ = labels
        self.n_clusters_ = n_clusters
        self.cluster_centers_ = centers
        self.autoencoder = autoencoder

# DipDeck

In [ ]:
#Datasets
#data, labels = MNIST()
#data, labels = FashionMNIST()
#data, labels = KMNIST()
data, labels = USPS()
#data, labels = OPTDIGITS()
#data, labels = PENDIGITS()


model = DipDeck()
model.fit(data)

print("K:", model.n_clusters_)
print("NMI:", nmi(labels, model.labels_))

100%|██████████| 6579383/6579383 [00:01<00:00, 4227208.00it/s]


100%|██████████| 1831726/1831726 [00:01<00:00, 1366286.41it/s]


K: 10
NMI: 0.8646958326774905
